In [ ]:
import pandas as pd
import xarray as xr

import config
from utils import processing

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
df = pd.read_csv(config.RAW_STORM_DB_PATH)
df.head()

,Storm,Life,Area,Initial UTC,Lon,Lat,Parent,Child,Accreted,u,...,LonInBounds,LatInBounds,in_domain,areaxgpm_mean,duration,duration_hr,speed,mean_speed,east_africa_time,eat_hours
0,3992,13,25,2014-01-02 14:15:00,38.900739,6.77762,[-999],[-999],[-999],0.15,...,True,True,True,NaN,0 days 03:15:00,3.25,2.134379,4.309462,2014-01-02 17:15:00,17.25
1,3992,13,40,2014-01-02 14:15:00,38.854092,6.77762,[-999],[-999],[-999],0.03,...,True,True,True,69.239998,0 days 03:15:00,3.25,4.353582,4.309462,2014-01-02 17:30:00,17.50
2,3992,13,52,2014-01-02 14:15:00,38.854092,6.77762,[-999],[-999],[-999],0.04,...,True,True,True,61.460004,0 days 03:15:00,3.25,0.901467,4.309462,2014-01-02 17:45:00,17.75
3,3992,13,68,2014-01-02 14:15:00,38.854092,6.77762,[-999],[-999],[-999],-1.03,...,True,True,True,NaN,0 days 03:15:00,3.25,7.018408,4.309462,2014-01-02 18:00:00,18.00
4,3992,13,81,2014-01-02 14:15:00,38.806765,6.74383,[-999],[-999],[-999],-0.54,...,True,True,True,NaN,0 days 03:15:00,3.25,3.344515,4.309462,2014-01-02 18:15:00,18.25


In [ ]:
df = processing.rename_columns(df, column_map=config.COL_RENAME_MAP)

In [ ]:
# load geopotential data and convert to height
geop, height = processing.load_geop_and_calc_elevation()

# load subgrid orography angle data
anor = xr.open_dataset(config.DATA_DIR / "std" / "anor.nc")

df = processing.get_orography_features(df, geop, height, anor)

In [ ]:
# select only the columns that are in the config
df = df[[col for col in config.DATASET_COL_NAMES if col in df.columns]]

In [ ]:
df.head()

,storm_id,timestamp,eat_hours,x,y,orography_height,anor,total_duration,mjo_phase,mjo_amplitude
0,3992,2014-01-02 14:15:00,17.25,38.900739,6.77762,2560.467285,-0.302176,3.25,5,0.43548
1,3992,2014-01-02 14:30:00,17.50,38.854092,6.77762,2518.048340,-0.347111,3.25,5,0.43548
2,3992,2014-01-02 14:45:00,17.75,38.854092,6.77762,2518.048340,-0.347111,3.25,5,0.43548
3,3992,2014-01-02 15:00:00,18.00,38.854092,6.77762,2518.048340,-0.347111,3.25,5,0.43548
4,3992,2014-01-02 15:15:00,18.25,38.806765,6.74383,2518.048340,-0.347111,3.25,5,0.43548


In [ ]:
df.to_csv(config.PROCESSED_DATASET_PATH, index=False)